In [1]:
# get_connected.py
def get_connected(market, trade_type):
    ''' get connected to ibkr
    Args: 
       (market) as string <'nse'> | <'snp'>
       (trade_type) as string <'live'> | <'paper'>
    Returns:
        (ib) object if successful
    '''
    
    ip = (market.upper(), trade_type.upper())
    
    #host dictionary
    hostdict = {('NSE', 'LIVE'): 3000,
                ('NSE', 'PAPER'): 3001,
                ('SNP', 'LIVE'): 1300,
                ('SNP', 'PAPER'): 1301,}
    
    host = hostdict[ip]
    
    cid = 1 # initialize clientId
    max_cid = 5 # maximum clientId allowed. max possible is 32

    for i in range(cid, max_cid):
        try:
            ib = IB().connect('127.0.0.1', host, clientId=i)
            
        except Exception as e:
            print(e) # print the error
            continue # go to next
            
        break # successful try
        
    return ib

In [8]:
# get_nses.py

import pandas as pd

blk = 50 # no of stocks in a block
exchange = 'NSE'
def get_nses(ib):
    '''Returns: list of nse underlying contracts
    '''
    tp = pd.read_html('https://www.tradeplusonline.com/Equity-Futures-Margin-Calculator.aspx')

    df_tp = tp[1][2:].iloc[:, :-1]
    df_tp = df_tp.iloc[:, [0,1,5]]
    df_tp.columns=['nseSymbol', 'lot', 'margin']

    cols = df_tp.columns.drop('nseSymbol')
    df_tp[cols] = df_tp[cols].apply(pd.to_numeric, errors='coerce') # convert lot and margin to numeric

    df_slm = df_tp.copy()

    # Truncate to 9 characters for ibSymbol
    df_slm['ibSymbol'] = df_slm.nseSymbol.str.slice(0,9)

    # nseSymbol to ibSymbol dictionary for conversion
    ntoi = {'M&M': 'MM', 'M&MFIN': 'MMFIN', 'L&TFH': 'LTFH', 'NIFTY': 'NIFTY50'}

    # remap ibSymbol, based on the dictionary
    df_slm.ibSymbol = df_slm.ibSymbol.replace(ntoi)

    # separate indexes and equities, eliminate discards from df_slm
    indexes = ['NIFTY50', 'BANKNIFTY']
    discards = ['NIFTYMID5', 'NIFTYIT', 'LUPIN']
    equities = sorted([s for s in df_slm.ibSymbol if s not in indexes+discards])

    symbols = equities+indexes

    cs = [Stock(s, exchange) if s in equities else Index(s, exchange) for s in symbols]

    qcs = ib.qualifyContracts(*cs) # qualified underlyings
    
    return qcs

In [17]:
from ib_insync import *
util.startLoop()

from os import listdir

from helper import get_connected

keep_pickles = True   # keep already pickled symbols

with get_connected('nse', 'live') as ib: 
    
    fspath = '../data/nse/' # path for pickles
    
    undContracts = get_nses(ib)
    tickers = ib.reqTickers(*undContracts)
    undPrices = {t.contract.symbol: t.marketPrice() for t in tickers} # {symbol: undPrice}
    
    util.logToFile(fspath+'_errors.log')  # create log file
    with open(fspath+'_errors.log', 'w'): # clear the previous log
        pass

    fs = listdir(fspath)
    optsList = [f[:-4] for f in fs if f[-3:] == 'pkl']

    if keep_pickles:
        contracts = [m for m in undContracts if m.symbol not in optsList]
        symbols = [c.symbol for c in contracts]
        prices = [undPrices[s] for s in symbols]